In [2]:
import polars as pl
import numpy as np
from math import ceil

In [3]:
df = pl.read_parquet('df_general_new.pq')

train_size = ceil(df.shape[0] * 0.8)
train_size

2480

In [12]:
train_indices = np.random.choice(np.arange(len(df)), train_size, replace=False)
val_indicies = np.array(list(set(np.arange(len(df))) - set(train_indices)))

In [4]:
df = df.drop(['query', 'chunk_text'])

In [14]:
df[train_indices].write_parquet('train_df.pq')
df[val_indicies].write_parquet('val_df.pq')

In [5]:
df_new = df.with_columns(query_rank = pl.col('query_embedding').rank(method='min'))
df_new = df_new.drop(["query_embedding","chunk_text_embedding","date","query_len","chunk_text_len"])
unique_rank = pl.Series(df_new.unique('query_rank').select('query_rank')).to_list()
train_ranks = np.random.choice(unique_rank, int(len(unique_rank) * 0.8), replace = False)
train_ranks
val_ranks = list(set(unique_rank) - set(train_ranks))

train_df = df_new.filter(pl.col('query_rank').is_in(train_ranks)).sort("query_rank")
val_df = df_new.filter(pl.col('query_rank').is_in(val_ranks)).sort("query_rank")
train_df.write_parquet('train_df.pq')
val_df.write_parquet('val_df.pq')